In [38]:
import tkinter as tk
from tkinter import filedialog, messagebox
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from PIL import Image
from datetime import datetime
import requests
import os
import openpyxl

# Set script_dir to the directory where your logo images and other files are located
script_dir = r"C:\Users\weber\Blisters"  # Update this path to your actual directory

# Verify that script_dir is correctly set
if not os.path.exists(script_dir):
    raise FileNotFoundError(f"The directory {script_dir} does not exist. Please update script_dir to the correct path.")

# Define the paths of the files based on script_dir
LAST_PATH_FILE = os.path.join(script_dir, 'last_path.txt')
EXCEL_FILE = os.path.join(script_dir, 'dados_cotacoes.xlsx')
COTACAO_SEQUENCE_FILE = os.path.join(script_dir, 'cotacao_sequence.txt')

# Function to obtain the current sequential number and increment it
def obter_numero_sequencial(filepath=COTACAO_SEQUENCE_FILE):
    try:
        with open(filepath, 'r') as file:
            numero_atual = int(file.read().strip())
    except FileNotFoundError:
        numero_atual = 3456  # Initial number if the file does not exist

    novo_numero = numero_atual + 1

    with open(filepath, 'w') as file:
        file.write(str(novo_numero))

    return numero_atual

# Function to fetch client data from CNPJ
def buscar_dados_cliente(cnpj):
    url = f"https://www.receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)
    if response.status_code == 200:
        dados = response.json()
        endereco = f'{dados.get("logradouro", "")}, {dados.get("numero", "")} - {dados.get("bairro", "")}'
        cidade_uf_cep = f'{dados.get("municipio", "")} - {dados.get("uf", "")}, CEP: {dados.get("cep", "")}'
        return {
            "razao_social": dados.get("nome", ""),
            "endereco": endereco,
            "cidade_uf_cep": cidade_uf_cep
        }
    else:
        return None

# Function to wrap text into multiple lines with a character limit
def wrap_text(text, char_limit=85):
    words = text.split()
    lines = []
    line = []
    for word in words:
        if len(' '.join(line + [word])) > char_limit:
            lines.append(' '.join(line))
            line = [word]
        else:
            line.append(word)
    if line:
        lines.append(' '.join(line))
    return lines

# Function to generate the PDF quotation
def gerar_cotacao_pdf(dados_cliente, itens, prazo_pagamento, frete_tipo, output_path, numero_sequencial, responsavel, empresa):
    if empresa == "Mokka":
        logo_filename = "Logo-Mokka-Sensors.jpg"
        footer_text = "Mokka Com. de Bens de Consumo Ltda - CNPJ: 21.220.932/0001-10"
        logo_percent = 0.35  # Adjust the logo size for Mokka here
        logo_y_position_adjustment = 2 * cm  # Adjust the logo y position for Mokka here
    else:
        logo_filename = "Logo-Moica.jpg"
        footer_text = "Moica Com. de Bens de Consumo Ltda - CNPJ: 42.044.083/0001-60"
        logo_percent = 0.25  # Adjust the logo size for Moica here
        logo_y_position_adjustment = 2.5 * cm  # Adjust the logo y position for Moica here

    # Build the full path to the logo image
    logo_path = os.path.join(script_dir, logo_filename)

    # Verify that the logo file exists
    if not os.path.exists(logo_path):
        messagebox.showerror("Erro", f"O arquivo de logotipo não foi encontrado: {logo_path}")
        return

    logo_image = Image.open(logo_path)
    original_width, original_height = logo_image.size

    # Page dimensions
    width, height = A4

    # Calculate the new width and height of the logo based on the percentage
    desired_width = width * logo_percent
    aspect_ratio = original_height / original_width
    desired_height = desired_width * aspect_ratio

    # Create the PDF canvas
    c = canvas.Canvas(output_path, pagesize=A4)
    page_number = 1

    # Function to draw the header on all pages
    def draw_header(c, page_number):
        logo_y_position = height - logo_y_position_adjustment - desired_height
        c.drawImage(logo_path, 2 * cm, logo_y_position, width=desired_width, height=desired_height)

        line_y_position = logo_y_position - 0.3 * cm
        c.setLineWidth(0.5)
        c.line(1 * cm, line_y_position, width - 1 * cm, line_y_position)

        data_criacao = datetime.now().strftime("%d/%m/%Y")
        c.setFont("Helvetica", 12)
        c.drawString(width - 5 * cm, height - 3 * cm, f"Data: {data_criacao}")

        c.setFont("Helvetica", 10)
        c.drawString(2 * cm, height - 4 * cm, f"Responsável: {responsavel['nome']}")
        c.drawString(2 * cm, height - 4.5 * cm, f"Contato: {responsavel['telefone']}")
        c.drawString(2 * cm, height - 5 * cm, f"{responsavel['email']}")
        c.drawString(2 * cm, height - 5.5 * cm, f"{responsavel['site1']}")
        c.drawString(2 * cm, height - 6 * cm, f"{responsavel['site2']}")

        c.drawCentredString(width / 2, 1.5 * cm, footer_text)
        c.drawString(width - 5 * cm, 1.5 * cm, f"Página {page_number}")

        footer_y_position = 1.5 * cm
        c.line(1 * cm, footer_y_position + 0.5 * cm, width - 1 * cm, footer_y_position + 0.5 * cm)

    draw_header(c, page_number)

    # Title with sequential number
    c.setFont("Helvetica-Bold", 16)
    c.drawString(2 * cm, height - 7 * cm, f"Cotação #{numero_sequencial}")

    # Client subtitle
    c.setFont("Helvetica-Bold", 14)
    c.drawString(2 * cm, height - 8.5 * cm, "Cliente")

    # Client data (Razão Social and Endereço on separate lines)
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, height - 9.5 * cm, f"Razão Social: {dados_cliente['razao_social']}")
    c.drawString(2 * cm, height - 10 * cm, f"Endereço: {dados_cliente['endereco']}")
    c.drawString(2 * cm, height - 10.5 * cm, f"{dados_cliente['cidade_uf_cep']}")

    # Fine line below the address
    c.setLineWidth(0.5)
    c.line(1 * cm, height - 11 * cm, width - 1 * cm, height - 11 * cm)

    current_y = height - 13 * cm

    for i, item in enumerate(itens, start=1):
        if current_y < 5 * cm:  # Adjust margin for page break
            c.showPage()
            page_number += 1
            draw_header(c, page_number)
            current_y = height - 7 * cm

        # Item subtitle
        c.setFont("Helvetica-Bold", 14)
        c.drawString(2 * cm, current_y, f"Item {i}")
        current_y -= 1 * cm

        # Product description
        c.setFont("Helvetica", 12)
        c.drawString(2 * cm, current_y, f"Produto: {item['produto']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, "Descrição:")
        current_y -= 0.5 * cm

        # Description
        descricao_lines = wrap_text(item['descricao'], char_limit=80)
        c.setFont("Helvetica", 12)
        for line in descricao_lines:
            if current_y < 2.5 * cm:  # Check if 2.5cm from footer
                c.showPage()
                page_number += 1
                draw_header(c, page_number)
                current_y = height - 7 * cm
                c.setFont("Helvetica", 12)
            c.drawString(2 * cm, current_y, line)
            current_y -= 0.6 * cm  # Adjust line spacing as needed

        current_y -= 0.5 * cm  # Blank line between Description and Unit Price
        c.drawString(2 * cm, current_y, f"Preço Unitário: R$ {item['preco_unitario'].replace('.', ',')}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Quantidade: {item['quantidade']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"NCM: {item['ncm']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Total: R$ {item['total'].replace('.', ',')}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Prazo de Entrega: {item['prazo_entrega']}")
        current_y -= 2 * cm  # Spacing between items

    if current_y < 5 * cm:
        c.showPage()
        page_number += 1
        draw_header(c, page_number)
        current_y = height - 7 * cm

    # Total of the Quotation
    total_cotacao = sum(float(item['total'].replace(',', '.')) for item in itens)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(2 * cm, current_y, f"TOTAL DA COTAÇÃO: R$ {total_cotacao:.2f}".replace('.', ','))
    current_y -= 1 * cm

    # Payment Terms
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, current_y, f"Prazo de Pagamento: {prazo_pagamento}")
    current_y -= 1 * cm

    # Freight Type
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, current_y, f"Frete: {frete_tipo}")
    current_y -= 1 * cm

    # Add line and footer with company name, CNPJ, and page number on all pages
    c.setFont("Helvetica", 10)
    footer_y_position = 1.5 * cm
    c.line(1 * cm, footer_y_position + 0.5 * cm, width - 1 * cm, footer_y_position + 0.5 * cm)
    c.drawCentredString(width / 2, footer_y_position, footer_text)
    c.drawString(width - 5 * cm, footer_y_position, f"Página {page_number}")

    # Add page with supply conditions
    c.showPage()
    page_number += 1
    draw_header(c, page_number)
    current_y = height - 9 * cm

    c.setFont("Helvetica-Bold", 14)
    c.drawString(2 * cm, current_y, "Condições de fornecimento")
    current_y -= 1 * cm

    c.setFont("Helvetica", 10)
    c.drawString(2 * cm, current_y, "1. A cotação é válida por 5 dias.")
    current_y -= 1 * cm
    c.drawString(2 * cm, current_y, "2. O prazo de entrega está sujeito a alterações.")
    current_y -= 1 * cm
    c.drawString(2 * cm, current_y, "3. Se o item não estiver disponível para pronta entrega, ele será fabricado ou importado especificamente para")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "atender à sua necessidade. Nesses casos, não aceitamos devoluções ou cancelamentos de pedidos.")
    current_y -= 1 * cm
    c.drawString(2 * cm, current_y, "4. Nossa empresa é optante pelo Simples Nacional, um regime tributário diferenciado e simplificado. Neste regime,")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "os impostos são pagos de forma unificada e estão todos inclusos no preço dos nossos produtos. Isso inclui tributos")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "federais, estaduais e municipais, como o Imposto de Renda, CSLL, PIS, COFINS, ICMS e ISS. Empresas que ")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "compram nossos produtos para revenda ou industrialização podem gerar crédito de ICMS com uma alíquota de")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "aproximadamente 4%. Esse crédito pode ser usado para compensar o ICMS em operações futuras.")

    # Add line and footer with company name, CNPJ, and page number on all pages
    c.setFont("Helvetica", 10)
    footer_y_position = 1.5 * cm
    c.line(1 * cm, footer_y_position + 0.5 * cm, width - 1 * cm, footer_y_position + 0.5 * cm)
    c.drawCentredString(width / 2, footer_y_position, footer_text)
    c.drawString(width - 5 * cm, footer_y_position, f"Página {page_number}")

    # Save the PDF
    c.save()

# Function to add data to Excel
def adicionar_dados_excel(dados_cliente, itens, numero_sequencial, cliente_tipo):
    if not os.path.exists(EXCEL_FILE):
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["Data da Cotação", "Número da Cotação", "Razão Social do Cliente", "CNPJ do Cliente", "Produto", "Total (item)", "Cliente Final ou Revenda"])
    else:
        wb = openpyxl.load_workbook(EXCEL_FILE)
        ws = wb.active

    data_cotacao = datetime.now().strftime("%d/%m/%Y")
    cnpj_cliente = entrada_cnpj.get()

    for item in itens:
        ws.append([
            data_cotacao,
            numero_sequencial,
            dados_cliente["razao_social"],
            cnpj_cliente,
            item["produto"],
            item["total"].replace('.', ','),
            cliente_tipo
        ])

    wb.save(EXCEL_FILE)

# Function to get the last saved path
def obter_ultimo_caminho():
    if os.path.exists(LAST_PATH_FILE):
        with open(LAST_PATH_FILE, 'r') as file:
            return file.read().strip()
    return ''

# Function to save the last path
def salvar_ultimo_caminho(caminho):
    with open(LAST_PATH_FILE, 'w') as file:
        file.write(caminho)

# Function to get the output path of the PDF file
def obter_caminho_saida(numero_sequencial):
    initial_dir = obter_ultimo_caminho() or os.getcwd()
    output_path = filedialog.asksaveasfilename(defaultextension=".pdf", title="Salvar Cotação Como",
                                               initialfile=f'Cotação {empresa_var.get()} {numero_sequencial}.pdf',
                                               filetypes=[("PDF files", "*.pdf")], initialdir=initial_dir)
    if output_path:
        salvar_ultimo_caminho(os.path.dirname(output_path))
    return output_path

# Function to validate decimal number input
def validate_decimal(P):
    if P == "":
        return True
    try:
        float(P.replace(',', '.'))
        return True
    except ValueError:
        return False

# Function to validate integer number input
def validate_integer(P):
    if P == "":
        return True
    try:
        int(P)
        return True
    except ValueError:
        return False

# Function to calculate the total value
def calcular_total(item_frame):
    try:
        preco_unitario = float(item_frame["preco_unitario"].get().replace(',', '.'))
        quantidade = int(item_frame["quantidade"].get())
        total = preco_unitario * quantidade
        item_frame["total"].config(state='normal')
        item_frame["total"].delete(0, tk.END)
        item_frame["total"].insert(0, f"{total:.2f}".replace('.', ','))
        item_frame["total"].config(state='readonly')
    except ValueError:
        item_frame["total"].config(state='normal')
        item_frame["total"].delete(0, tk.END)
        item_frame["total"].insert(0, "0,00")
        item_frame["total"].config(state='readonly')
    calcular_total_cotacao()

# Function to calculate the total of the quotation
def calcular_total_cotacao():
    total = 0
    for item_frame in item_frames:
        try:
            total += float(item_frame["total"].get().replace(',', '.'))
        except ValueError:
            pass
    entrada_total_cotacao.config(state='normal')
    entrada_total_cotacao.delete(0, tk.END)
    entrada_total_cotacao.insert(0, f"{total:.2f}".replace('.', ','))
    entrada_total_cotacao.config(state='readonly')

# Function to generate the quotation when the button is clicked
def gerar_cotacao():
    cnpj_cliente = entrada_cnpj.get()
    prazo_pagamento = entrada_prazo_pagamento.get()
    frete_tipo = frete_var.get()
    numero_sequencial = obter_numero_sequencial()
    responsavel_nome = responsavel_var.get()
    cliente_tipo = cliente_tipo_var.get()
    empresa = empresa_var.get()

    # Data of the person responsible for the quotation
    responsaveis = {
        "Weber Melo": {
            "nome": "Weber Melo",
            "telefone": "(11) 98477-9490",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        },
        "Thiago Velicev": {
            "nome": "Thiago Velicev",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        },
        "Giulia Armelin": {
            "nome": "Giulia Armelin",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        },
        "Letícia Casale": {
            "nome": "Letícia Casale",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        }
    }

    responsavel = responsaveis.get(responsavel_nome)
    output_path = obter_caminho_saida(numero_sequencial)

    if not all([cnpj_cliente, prazo_pagamento, frete_tipo, output_path]):
        messagebox.showerror("Erro", "Todos os campos devem ser preenchidos.")
        return

    dados_cliente = buscar_dados_cliente(cnpj_cliente)

    if not dados_cliente:
        messagebox.showerror("Erro", "Não foi possível obter os dados do cliente.")
        return

    itens = []
    for item_frame in item_frames:
        produto = item_frame["produto"].get()
        descricao = item_frame["descricao"].get()
        preco_unitario = item_frame["preco_unitario"].get()
        quantidade = item_frame["quantidade"].get()
        ncm = item_frame["ncm"].get()
        total = item_frame["total"].get()
        prazo_entrega = item_frame["prazo_entrega"].get()

        if not all([produto, descricao, preco_unitario, quantidade, ncm, total, prazo_entrega]):
            messagebox.showerror("Erro", "Todos os campos dos itens devem ser preenchidos.")
            return

        itens.append({
            "produto": produto,
            "descricao": descricao,
            "preco_unitario": preco_unitario,
            "quantidade": quantidade,
            "ncm": ncm,
            "total": total,
            "prazo_entrega": prazo_entrega
        })

    gerar_cotacao_pdf(dados_cliente, itens, prazo_pagamento, frete_tipo, output_path, numero_sequencial, responsavel, empresa)
    adicionar_dados_excel(dados_cliente, itens, numero_sequencial, cliente_tipo)
    messagebox.showinfo("Sucesso", f"Cotação salva em: {output_path}")

# Function to add a new item
def adicionar_item():
    item_count = len(item_frames) + 1

    item_frame = {
        "label_item": tk.Label(items_frame, text=f"Item {item_count}"),
        "produto_label": tk.Label(items_frame, text="Produto:"),
        "produto": tk.Entry(items_frame, width=50),
        "descricao_label": tk.Label(items_frame, text="Descrição:"),
        "descricao": tk.Entry(items_frame, width=80),  # Increase the width of the description field
        "preco_unitario_label": tk.Label(items_frame, text="Preço Unitário: R$"),
        "preco_unitario": tk.Entry(items_frame, width=20, validate="key", validatecommand=(janela.register(validate_decimal), "%P")),
        "quantidade_label": tk.Label(items_frame, text="Quantidade:"),
        "quantidade": tk.Entry(items_frame, width=20, validate="key", validatecommand=(janela.register(validate_integer), "%P")),
        "ncm_label": tk.Label(items_frame, text="NCM:"),
        "ncm": tk.Entry(items_frame, width=20),
        "total_label": tk.Label(items_frame, text="Total: R$"),
        "total": tk.Entry(items_frame, width=20, state='readonly'),
        "prazo_entrega_label": tk.Label(items_frame, text="Prazo de Entrega:"),
        "prazo_entrega": tk.Entry(items_frame, width=20),
        "botao_remover": None  # Initialize with None
    }

    row = 4 + len(item_frames) * 10

    item_frame["label_item"].grid(row=row, column=0, padx=10, pady=5, columnspan=2, sticky='w')
    item_frame["produto_label"].grid(row=row+1, column=0, padx=10, pady=5, sticky='w')
    item_frame["produto"].grid(row=row+1, column=1, padx=10, pady=5, sticky='w')
    item_frame["descricao_label"].grid(row=row+2, column=0, padx=10, pady=5, sticky='w')
    item_frame["descricao"].grid(row=row+2, column=1, padx=10, pady=5, sticky='w')
    item_frame["preco_unitario_label"].grid(row=row+3, column=0, padx=10, pady=5, sticky='w')
    item_frame["preco_unitario"].grid(row=row+3, column=1, padx=10, pady=5, sticky='w')
    item_frame["quantidade_label"].grid(row=row+4, column=0, padx=10, pady=5, sticky='w')
    item_frame["quantidade"].grid(row=row+4, column=1, padx=10, pady=5, sticky='w')
    item_frame["ncm_label"].grid(row=row+5, column=0, padx=10, pady=5, sticky='w')
    item_frame["ncm"].grid(row=row+5, column=1, padx=10, pady=5, sticky='w')
    item_frame["total_label"].grid(row=row+6, column=0, padx=10, pady=5, sticky='w')
    item_frame["total"].grid(row=row+6, column=1, padx=10, pady=5, sticky='w')
    item_frame["prazo_entrega_label"].grid(row=row+7, column=0, padx=10, pady=5, sticky='w')
    item_frame["prazo_entrega"].grid(row=row+7, column=1, padx=10, pady=5, sticky='w')

    # Add the remove button after all other widgets have been defined
    item_frame["botao_remover"] = tk.Button(items_frame, text="Remover", command=lambda frame=item_frame: remover_item(frame))
    item_frame["botao_remover"].grid(row=row+8, column=0, columnspan=2, pady=5, sticky='w')

    item_frame["preco_unitario"].bind("<KeyRelease>", lambda event, frame=item_frame: calcular_total(frame))
    item_frame["quantidade"].bind("<KeyRelease>", lambda event, frame=item_frame: calcular_total(frame))

    item_frames.append(item_frame)
    items_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

# Function to remove an item
def remover_item(item_frame):
    for widget in item_frame.values():
        if widget is not None:
            widget.grid_forget()
            widget.destroy()
    item_frames.remove(item_frame)
    recalcular_itens()

def recalcular_itens():
    for i, item_frame in enumerate(item_frames):
        item_frame["label_item"].config(text=f"Item {i + 1}")
        item_frame["label_item"].grid(row=4 + i * 10, column=0, padx=10, pady=5, columnspan=2, sticky='w')
        item_frame["produto_label"].grid(row=5 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["produto"].grid(row=5 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["descricao_label"].grid(row=6 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["descricao"].grid(row=6 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["preco_unitario_label"].grid(row=7 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["preco_unitario"].grid(row=7 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["quantidade_label"].grid(row=8 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["quantidade"].grid(row=8 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["ncm_label"].grid(row=9 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["ncm"].grid(row=9 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["total_label"].grid(row=10 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["total"].grid(row=10 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["prazo_entrega_label"].grid(row=11 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["prazo_entrega"].grid(row=11 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["botao_remover"].grid(row=12 + i * 10, column=0, columnspan=2, pady=5, sticky='w')
    calcular_total_cotacao()
    items_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

# Create the graphical interface
janela = tk.Tk()
janela.title("Gerador de Cotações")
janela.geometry("1200x800")  # Increase the size of the graphical interface window

# Configure the grid for the main window
janela.grid_rowconfigure(0, weight=1)
janela.grid_columnconfigure(0, weight=1)

# Main frame for the scrollbar
main_frame = tk.Frame(janela)
main_frame.grid(row=0, column=0, sticky="nsew")

tk_canvas = tk.Canvas(main_frame, width=1200, height=800)
tk_canvas.grid(row=0, column=0, sticky="nsew")

scrollbar = tk.Scrollbar(main_frame, orient="vertical", command=tk_canvas.yview)
scrollbar.grid(row=0, column=2, sticky="ns")  # Move scrollbar to column 2

tk_canvas.configure(yscrollcommand=scrollbar.set)

items_frame = tk.Frame(tk_canvas, padx=10, pady=10)  # Add padding
tk_canvas.create_window((0, 0), window=items_frame, anchor="nw")

items_frame.bind(
    "<Configure>",
    lambda e: tk_canvas.configure(
        scrollregion=tk_canvas.bbox("all")
    )
)

# Selection of the company (Mokka or Moica)
tk.Label(items_frame, text="Empresa").grid(row=0, column=0, padx=10, pady=5, sticky='w')
empresa_var = tk.StringVar(value="Mokka")
empresa_options = ["Mokka", "Moica"]
empresa_menu = tk.OptionMenu(items_frame, empresa_var, *empresa_options)
empresa_menu.grid(row=0, column=1, padx=10, pady=5, sticky='w')

# Add fields to the graphical interface
tk.Label(items_frame, text="CNPJ do Cliente").grid(row=1, column=0, padx=10, pady=5, sticky='w')
entrada_cnpj = tk.Entry(items_frame, width=50)
entrada_cnpj.grid(row=1, column=1, padx=10, pady=5, sticky='w')

# Selection of the person responsible for the quotation
tk.Label(items_frame, text="Responsável pela Cotação").grid(row=2, column=0, padx=10, pady=5, sticky='w')
responsavel_var = tk.StringVar()
responsavel_options = ["Weber Melo", "Thiago Velicev", "Giulia Armelin", "Letícia Casale"]
responsavel_menu = tk.OptionMenu(items_frame, responsavel_var, *responsavel_options)
responsavel_menu.grid(row=2, column=1, padx=10, pady=5, sticky='w')

# Selection of the client type (Final Client or Resale)
tk.Label(items_frame, text="Tipo de Cliente").grid(row=3, column=0, padx=10, pady=5, sticky='w')
cliente_tipo_var = tk.StringVar()
cliente_tipo_options = ["Cliente Final", "Revenda"]
cliente_tipo_menu = tk.OptionMenu(items_frame, cliente_tipo_var, *cliente_tipo_options)
cliente_tipo_menu.grid(row=3, column=1, padx=10, pady=5, sticky='w')

item_frames = []
adicionar_item()

tk.Label(items_frame, text="TOTAL DA COTAÇÃO").grid(row=100, column=0, padx=10, pady=5, sticky='w')
entrada_total_cotacao = tk.Entry(items_frame, width=20, state='readonly')
entrada_total_cotacao.grid(row=100, column=1, padx=10, pady=5, sticky='w')

tk.Label(items_frame, text="Prazo de Pagamento").grid(row=101, column=0, padx=10, pady=5, sticky='w')
entrada_prazo_pagamento = tk.Entry(items_frame, width=20)
entrada_prazo_pagamento.grid(row=101, column=1, padx=10, pady=5, sticky='w')

# Selection of the freight type
tk.Label(items_frame, text="Frete").grid(row=102, column=0, padx=10, pady=5, sticky='w')
frete_var = tk.StringVar(value="FOB")
tk.Radiobutton(items_frame, text="FOB", variable=frete_var, value="FOB").grid(row=102, column=1, padx=10, pady=5, sticky='w')
tk.Radiobutton(items_frame, text="CIF", variable=frete_var, value="CIF").grid(row=102, column=1, padx=100, pady=5, sticky='w')

# Spacing between fields and scrollbar
tk.Canvas(items_frame, width=cm).grid(row=0, column=2, rowspan=103)

botao_adicionar_item = tk.Button(items_frame, text="Adicionar Item", command=adicionar_item)
botao_adicionar_item.grid(row=104, column=0, columnspan=2, pady=10, sticky='w')

botao_gerar = tk.Button(items_frame, text="Gerar Cotação", command=gerar_cotacao)
botao_gerar.grid(row=105, column=0, columnspan=2, pady=20, sticky='w')

janela.mainloop()
import tkinter as tk
from tkinter import filedialog, messagebox
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from PIL import Image
from datetime import datetime
import requests
import os
import openpyxl

# Set script_dir to the directory where your logo images and other files are located
script_dir = r"C:\Users\weber\Blisters"  # Update this path to your actual directory

# Verify that script_dir is correctly set
if not os.path.exists(script_dir):
    raise FileNotFoundError(f"The directory {script_dir} does not exist. Please update script_dir to the correct path.")

# Define the paths of the files based on script_dir
LAST_PATH_FILE = os.path.join(script_dir, 'last_path.txt')
EXCEL_FILE = os.path.join(script_dir, 'dados_cotacoes.xlsx')
COTACAO_SEQUENCE_FILE = os.path.join(script_dir, 'cotacao_sequence.txt')

# Function to obtain the current sequential number and increment it
def obter_numero_sequencial(filepath=COTACAO_SEQUENCE_FILE):
    try:
        with open(filepath, 'r') as file:
            numero_atual = int(file.read().strip())
    except FileNotFoundError:
        numero_atual = 3456  # Initial number if the file does not exist

    novo_numero = numero_atual + 1

    with open(filepath, 'w') as file:
        file.write(str(novo_numero))

    return numero_atual

# Function to fetch client data from CNPJ
def buscar_dados_cliente(cnpj):
    url = f"https://www.receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)
    if response.status_code == 200:
        dados = response.json()
        endereco = f'{dados.get("logradouro", "")}, {dados.get("numero", "")} - {dados.get("bairro", "")}'
        cidade_uf_cep = f'{dados.get("municipio", "")} - {dados.get("uf", "")}, CEP: {dados.get("cep", "")}'
        return {
            "razao_social": dados.get("nome", ""),
            "endereco": endereco,
            "cidade_uf_cep": cidade_uf_cep
        }
    else:
        return None

# Function to wrap text into multiple lines with a character limit
def wrap_text(text, char_limit=85):
    words = text.split()
    lines = []
    line = []
    for word in words:
        if len(' '.join(line + [word])) > char_limit:
            lines.append(' '.join(line))
            line = [word]
        else:
            line.append(word)
    if line:
        lines.append(' '.join(line))
    return lines

# Function to generate the PDF quotation
def gerar_cotacao_pdf(dados_cliente, itens, prazo_pagamento, frete_tipo, output_path, numero_sequencial, responsavel, empresa):
    if empresa == "Mokka":
        logo_filename = "Logo-Mokka-Sensors.jpg"
        footer_text = "Mokka Com. de Bens de Consumo Ltda - CNPJ: 21.220.932/0001-10"
        logo_percent = 0.35  # Adjust the logo size for Mokka here
        logo_y_position_adjustment = 2 * cm  # Adjust the logo y position for Mokka here
    else:
        logo_filename = "Logo-Moica.jpg"
        footer_text = "Moica Com. de Bens de Consumo Ltda - CNPJ: 42.044.083/0001-60"
        logo_percent = 0.25  # Adjust the logo size for Moica here
        logo_y_position_adjustment = 2.5 * cm  # Adjust the logo y position for Moica here

    # Build the full path to the logo image
    logo_path = os.path.join(script_dir, logo_filename)

    # Verify that the logo file exists
    if not os.path.exists(logo_path):
        messagebox.showerror("Erro", f"O arquivo de logotipo não foi encontrado: {logo_path}")
        return

    logo_image = Image.open(logo_path)
    original_width, original_height = logo_image.size

    # Page dimensions
    width, height = A4

    # Calculate the new width and height of the logo based on the percentage
    desired_width = width * logo_percent
    aspect_ratio = original_height / original_width
    desired_height = desired_width * aspect_ratio

    # Create the PDF canvas
    c = canvas.Canvas(output_path, pagesize=A4)
    page_number = 1

    # Function to draw the header on all pages
    def draw_header(c, page_number):
        logo_y_position = height - logo_y_position_adjustment - desired_height
        c.drawImage(logo_path, 2 * cm, logo_y_position, width=desired_width, height=desired_height)

        line_y_position = logo_y_position - 0.3 * cm
        c.setLineWidth(0.5)
        c.line(1 * cm, line_y_position, width - 1 * cm, line_y_position)

        data_criacao = datetime.now().strftime("%d/%m/%Y")
        c.setFont("Helvetica", 12)
        c.drawString(width - 5 * cm, height - 3 * cm, f"Data: {data_criacao}")

        c.setFont("Helvetica", 10)
        c.drawString(2 * cm, height - 4 * cm, f"Responsável: {responsavel['nome']}")
        c.drawString(2 * cm, height - 4.5 * cm, f"Contato: {responsavel['telefone']}")
        c.drawString(2 * cm, height - 5 * cm, f"{responsavel['email']}")
        c.drawString(2 * cm, height - 5.5 * cm, f"{responsavel['site1']}")
        c.drawString(2 * cm, height - 6 * cm, f"{responsavel['site2']}")

        c.drawCentredString(width / 2, 1.5 * cm, footer_text)
        c.drawString(width - 5 * cm, 1.5 * cm, f"Página {page_number}")

        footer_y_position = 1.5 * cm
        c.line(1 * cm, footer_y_position + 0.5 * cm, width - 1 * cm, footer_y_position + 0.5 * cm)

    draw_header(c, page_number)

    # Title with sequential number
    c.setFont("Helvetica-Bold", 16)
    c.drawString(2 * cm, height - 7 * cm, f"Cotação #{numero_sequencial}")

    # Client subtitle
    c.setFont("Helvetica-Bold", 14)
    c.drawString(2 * cm, height - 8.5 * cm, "Cliente")

    # Client data (Razão Social and Endereço on separate lines)
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, height - 9.5 * cm, f"Razão Social: {dados_cliente['razao_social']}")
    c.drawString(2 * cm, height - 10 * cm, f"Endereço: {dados_cliente['endereco']}")
    c.drawString(2 * cm, height - 10.5 * cm, f"{dados_cliente['cidade_uf_cep']}")

    # Fine line below the address
    c.setLineWidth(0.5)
    c.line(1 * cm, height - 11 * cm, width - 1 * cm, height - 11 * cm)

    current_y = height - 13 * cm

    for i, item in enumerate(itens, start=1):
        if current_y < 5 * cm:  # Adjust margin for page break
            c.showPage()
            page_number += 1
            draw_header(c, page_number)
            current_y = height - 7 * cm

        # Item subtitle
        c.setFont("Helvetica-Bold", 14)
        c.drawString(2 * cm, current_y, f"Item {i}")
        current_y -= 1 * cm

        # Product description
        c.setFont("Helvetica", 12)
        c.drawString(2 * cm, current_y, f"Produto: {item['produto']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, "Descrição:")
        current_y -= 0.5 * cm

        # Description
        descricao_lines = wrap_text(item['descricao'], char_limit=80)
        c.setFont("Helvetica", 12)
        for line in descricao_lines:
            if current_y < 2.5 * cm:  # Check if 2.5cm from footer
                c.showPage()
                page_number += 1
                draw_header(c, page_number)
                current_y = height - 7 * cm
                c.setFont("Helvetica", 12)
            c.drawString(2 * cm, current_y, line)
            current_y -= 0.6 * cm  # Adjust line spacing as needed

        current_y -= 0.5 * cm  # Blank line between Description and Unit Price
        c.drawString(2 * cm, current_y, f"Preço Unitário: R$ {item['preco_unitario'].replace('.', ',')}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Quantidade: {item['quantidade']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"NCM: {item['ncm']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Total: R$ {item['total'].replace('.', ',')}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Prazo de Entrega: {item['prazo_entrega']}")
        current_y -= 2 * cm  # Spacing between items

    if current_y < 5 * cm:
        c.showPage()
        page_number += 1
        draw_header(c, page_number)
        current_y = height - 7 * cm

    # Total of the Quotation
    total_cotacao = sum(float(item['total'].replace(',', '.')) for item in itens)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(2 * cm, current_y, f"TOTAL DA COTAÇÃO: R$ {total_cotacao:.2f}".replace('.', ','))
    current_y -= 1 * cm

    # Payment Terms
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, current_y, f"Prazo de Pagamento: {prazo_pagamento}")
    current_y -= 1 * cm

    # Freight Type
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, current_y, f"Frete: {frete_tipo}")
    current_y -= 1 * cm

    # Add line and footer with company name, CNPJ, and page number on all pages
    c.setFont("Helvetica", 10)
    footer_y_position = 1.5 * cm
    c.line(1 * cm, footer_y_position + 0.5 * cm, width - 1 * cm, footer_y_position + 0.5 * cm)
    c.drawCentredString(width / 2, footer_y_position, footer_text)
    c.drawString(width - 5 * cm, footer_y_position, f"Página {page_number}")

    # Add page with supply conditions
    c.showPage()
    page_number += 1
    draw_header(c, page_number)
    current_y = height - 9 * cm

    c.setFont("Helvetica-Bold", 14)
    c.drawString(2 * cm, current_y, "Condições de fornecimento")
    current_y -= 1 * cm

    c.setFont("Helvetica", 10)
    c.drawString(2 * cm, current_y, "1. A cotação é válida por 5 dias.")
    current_y -= 1 * cm
    c.drawString(2 * cm, current_y, "2. O prazo de entrega está sujeito a alterações.")
    current_y -= 1 * cm
    c.drawString(2 * cm, current_y, "3. Se o item não estiver disponível para pronta entrega, ele será fabricado ou importado especificamente para")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "atender à sua necessidade. Nesses casos, não aceitamos devoluções ou cancelamentos de pedidos.")
    current_y -= 1 * cm
    c.drawString(2 * cm, current_y, "4. Nossa empresa é optante pelo Simples Nacional, um regime tributário diferenciado e simplificado. Neste regime,")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "os impostos são pagos de forma unificada e estão todos inclusos no preço dos nossos produtos. Isso inclui tributos")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "federais, estaduais e municipais, como o Imposto de Renda, CSLL, PIS, COFINS, ICMS e ISS. Empresas que ")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "compram nossos produtos para revenda ou industrialização podem gerar crédito de ICMS com uma alíquota de")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "aproximadamente 4%. Esse crédito pode ser usado para compensar o ICMS em operações futuras.")

    # Add line and footer with company name, CNPJ, and page number on all pages
    c.setFont("Helvetica", 10)
    footer_y_position = 1.5 * cm
    c.line(1 * cm, footer_y_position + 0.5 * cm, width - 1 * cm, footer_y_position + 0.5 * cm)
    c.drawCentredString(width / 2, footer_y_position, footer_text)
    c.drawString(width - 5 * cm, footer_y_position, f"Página {page_number}")

    # Save the PDF
    c.save()

# Function to add data to Excel
def adicionar_dados_excel(dados_cliente, itens, numero_sequencial, cliente_tipo):
    if not os.path.exists(EXCEL_FILE):
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["Data da Cotação", "Número da Cotação", "Razão Social do Cliente", "CNPJ do Cliente", "Produto", "Total (item)", "Cliente Final ou Revenda"])
    else:
        wb = openpyxl.load_workbook(EXCEL_FILE)
        ws = wb.active

    data_cotacao = datetime.now().strftime("%d/%m/%Y")
    cnpj_cliente = entrada_cnpj.get()

    for item in itens:
        ws.append([
            data_cotacao,
            numero_sequencial,
            dados_cliente["razao_social"],
            cnpj_cliente,
            item["produto"],
            item["total"].replace('.', ','),
            cliente_tipo
        ])

    wb.save(EXCEL_FILE)

# Function to get the last saved path
def obter_ultimo_caminho():
    if os.path.exists(LAST_PATH_FILE):
        with open(LAST_PATH_FILE, 'r') as file:
            return file.read().strip()
    return ''

# Function to save the last path
def salvar_ultimo_caminho(caminho):
    with open(LAST_PATH_FILE, 'w') as file:
        file.write(caminho)

# Function to get the output path of the PDF file
def obter_caminho_saida(numero_sequencial):
    initial_dir = obter_ultimo_caminho() or os.getcwd()
    output_path = filedialog.asksaveasfilename(defaultextension=".pdf", title="Salvar Cotação Como",
                                               initialfile=f'Cotação {empresa_var.get()} {numero_sequencial}.pdf',
                                               filetypes=[("PDF files", "*.pdf")], initialdir=initial_dir)
    if output_path:
        salvar_ultimo_caminho(os.path.dirname(output_path))
    return output_path

# Function to validate decimal number input
def validate_decimal(P):
    if P == "":
        return True
    try:
        float(P.replace(',', '.'))
        return True
    except ValueError:
        return False

# Function to validate integer number input
def validate_integer(P):
    if P == "":
        return True
    try:
        int(P)
        return True
    except ValueError:
        return False

# Function to calculate the total value
def calcular_total(item_frame):
    try:
        preco_unitario = float(item_frame["preco_unitario"].get().replace(',', '.'))
        quantidade = int(item_frame["quantidade"].get())
        total = preco_unitario * quantidade
        item_frame["total"].config(state='normal')
        item_frame["total"].delete(0, tk.END)
        item_frame["total"].insert(0, f"{total:.2f}".replace('.', ','))
        item_frame["total"].config(state='readonly')
    except ValueError:
        item_frame["total"].config(state='normal')
        item_frame["total"].delete(0, tk.END)
        item_frame["total"].insert(0, "0,00")
        item_frame["total"].config(state='readonly')
    calcular_total_cotacao()

# Function to calculate the total of the quotation
def calcular_total_cotacao():
    total = 0
    for item_frame in item_frames:
        try:
            total += float(item_frame["total"].get().replace(',', '.'))
        except ValueError:
            pass
    entrada_total_cotacao.config(state='normal')
    entrada_total_cotacao.delete(0, tk.END)
    entrada_total_cotacao.insert(0, f"{total:.2f}".replace('.', ','))
    entrada_total_cotacao.config(state='readonly')

# Function to generate the quotation when the button is clicked
def gerar_cotacao():
    cnpj_cliente = entrada_cnpj.get()
    prazo_pagamento = entrada_prazo_pagamento.get()
    frete_tipo = frete_var.get()
    numero_sequencial = obter_numero_sequencial()
    responsavel_nome = responsavel_var.get()
    cliente_tipo = cliente_tipo_var.get()
    empresa = empresa_var.get()

    # Data of the person responsible for the quotation
    responsaveis = {
        "Weber Melo": {
            "nome": "Weber Melo",
            "telefone": "(11) 98477-9490",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        },
        "Thiago Velicev": {
            "nome": "Thiago Velicev",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        },
        "Giulia Armelin": {
            "nome": "Giulia Armelin",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        },
        "Letícia Casale": {
            "nome": "Letícia Casale",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br"
        }
    }

    responsavel = responsaveis.get(responsavel_nome)
    output_path = obter_caminho_saida(numero_sequencial)

    if not all([cnpj_cliente, prazo_pagamento, frete_tipo, output_path]):
        messagebox.showerror("Erro", "Todos os campos devem ser preenchidos.")
        return

    dados_cliente = buscar_dados_cliente(cnpj_cliente)

    if not dados_cliente:
        messagebox.showerror("Erro", "Não foi possível obter os dados do cliente.")
        return

    itens = []
    for item_frame in item_frames:
        produto = item_frame["produto"].get()
        descricao = item_frame["descricao"].get()
        preco_unitario = item_frame["preco_unitario"].get()
        quantidade = item_frame["quantidade"].get()
        ncm = item_frame["ncm"].get()
        total = item_frame["total"].get()
        prazo_entrega = item_frame["prazo_entrega"].get()

        if not all([produto, descricao, preco_unitario, quantidade, ncm, total, prazo_entrega]):
            messagebox.showerror("Erro", "Todos os campos dos itens devem ser preenchidos.")
            return

        itens.append({
            "produto": produto,
            "descricao": descricao,
            "preco_unitario": preco_unitario,
            "quantidade": quantidade,
            "ncm": ncm,
            "total": total,
            "prazo_entrega": prazo_entrega
        })

    gerar_cotacao_pdf(dados_cliente, itens, prazo_pagamento, frete_tipo, output_path, numero_sequencial, responsavel, empresa)
    adicionar_dados_excel(dados_cliente, itens, numero_sequencial, cliente_tipo)
    messagebox.showinfo("Sucesso", f"Cotação salva em: {output_path}")

# Function to add a new item
def adicionar_item():
    item_count = len(item_frames) + 1

    item_frame = {
        "label_item": tk.Label(items_frame, text=f"Item {item_count}"),
        "produto_label": tk.Label(items_frame, text="Produto:"),
        "produto": tk.Entry(items_frame, width=50),
        "descricao_label": tk.Label(items_frame, text="Descrição:"),
        "descricao": tk.Entry(items_frame, width=80),  # Increase the width of the description field
        "preco_unitario_label": tk.Label(items_frame, text="Preço Unitário: R$"),
        "preco_unitario": tk.Entry(items_frame, width=20, validate="key", validatecommand=(janela.register(validate_decimal), "%P")),
        "quantidade_label": tk.Label(items_frame, text="Quantidade:"),
        "quantidade": tk.Entry(items_frame, width=20, validate="key", validatecommand=(janela.register(validate_integer), "%P")),
        "ncm_label": tk.Label(items_frame, text="NCM:"),
        "ncm": tk.Entry(items_frame, width=20),
        "total_label": tk.Label(items_frame, text="Total: R$"),
        "total": tk.Entry(items_frame, width=20, state='readonly'),
        "prazo_entrega_label": tk.Label(items_frame, text="Prazo de Entrega:"),
        "prazo_entrega": tk.Entry(items_frame, width=20),
        "botao_remover": None  # Initialize with None
    }

    row = 4 + len(item_frames) * 10

    item_frame["label_item"].grid(row=row, column=0, padx=10, pady=5, columnspan=2, sticky='w')
    item_frame["produto_label"].grid(row=row+1, column=0, padx=10, pady=5, sticky='w')
    item_frame["produto"].grid(row=row+1, column=1, padx=10, pady=5, sticky='w')
    item_frame["descricao_label"].grid(row=row+2, column=0, padx=10, pady=5, sticky='w')
    item_frame["descricao"].grid(row=row+2, column=1, padx=10, pady=5, sticky='w')
    item_frame["preco_unitario_label"].grid(row=row+3, column=0, padx=10, pady=5, sticky='w')
    item_frame["preco_unitario"].grid(row=row+3, column=1, padx=10, pady=5, sticky='w')
    item_frame["quantidade_label"].grid(row=row+4, column=0, padx=10, pady=5, sticky='w')
    item_frame["quantidade"].grid(row=row+4, column=1, padx=10, pady=5, sticky='w')
    item_frame["ncm_label"].grid(row=row+5, column=0, padx=10, pady=5, sticky='w')
    item_frame["ncm"].grid(row=row+5, column=1, padx=10, pady=5, sticky='w')
    item_frame["total_label"].grid(row=row+6, column=0, padx=10, pady=5, sticky='w')
    item_frame["total"].grid(row=row+6, column=1, padx=10, pady=5, sticky='w')
    item_frame["prazo_entrega_label"].grid(row=row+7, column=0, padx=10, pady=5, sticky='w')
    item_frame["prazo_entrega"].grid(row=row+7, column=1, padx=10, pady=5, sticky='w')

    # Add the remove button after all other widgets have been defined
    item_frame["botao_remover"] = tk.Button(items_frame, text="Remover", command=lambda frame=item_frame: remover_item(frame))
    item_frame["botao_remover"].grid(row=row+8, column=0, columnspan=2, pady=5, sticky='w')

    item_frame["preco_unitario"].bind("<KeyRelease>", lambda event, frame=item_frame: calcular_total(frame))
    item_frame["quantidade"].bind("<KeyRelease>", lambda event, frame=item_frame: calcular_total(frame))

    item_frames.append(item_frame)
    items_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

# Function to remove an item
def remover_item(item_frame):
    for widget in item_frame.values():
        if widget is not None:
            widget.grid_forget()
            widget.destroy()
    item_frames.remove(item_frame)
    recalcular_itens()

def recalcular_itens():
    for i, item_frame in enumerate(item_frames):
        item_frame["label_item"].config(text=f"Item {i + 1}")
        item_frame["label_item"].grid(row=4 + i * 10, column=0, padx=10, pady=5, columnspan=2, sticky='w')
        item_frame["produto_label"].grid(row=5 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["produto"].grid(row=5 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["descricao_label"].grid(row=6 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["descricao"].grid(row=6 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["preco_unitario_label"].grid(row=7 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["preco_unitario"].grid(row=7 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["quantidade_label"].grid(row=8 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["quantidade"].grid(row=8 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["ncm_label"].grid(row=9 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["ncm"].grid(row=9 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["total_label"].grid(row=10 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["total"].grid(row=10 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["prazo_entrega_label"].grid(row=11 + i * 10, column=0, padx=10, pady=5, sticky='w')
        item_frame["prazo_entrega"].grid(row=11 + i * 10, column=1, padx=10, pady=5, sticky='w')
        item_frame["botao_remover"].grid(row=12 + i * 10, column=0, columnspan=2, pady=5, sticky='w')
    calcular_total_cotacao()
    items_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

# Create the graphical interface
janela = tk.Tk()
janela.title("Gerador de Cotações")
janela.geometry("1200x800")  # Increase the size of the graphical interface window

# Configure the grid for the main window
janela.grid_rowconfigure(0, weight=1)
janela.grid_columnconfigure(0, weight=1)

# Main frame for the scrollbar
main_frame = tk.Frame(janela)
main_frame.grid(row=0, column=0, sticky="nsew")

tk_canvas = tk.Canvas(main_frame, width=1200, height=800)
tk_canvas.grid(row=0, column=0, sticky="nsew")

scrollbar = tk.Scrollbar(main_frame, orient="vertical", command=tk_canvas.yview)
scrollbar.grid(row=0, column=2, sticky="ns")  # Move scrollbar to column 2

tk_canvas.configure(yscrollcommand=scrollbar.set)

items_frame = tk.Frame(tk_canvas, padx=10, pady=10)  # Add padding
tk_canvas.create_window((0, 0), window=items_frame, anchor="nw")

items_frame.bind(
    "<Configure>",
    lambda e: tk_canvas.configure(
        scrollregion=tk_canvas.bbox("all")
    )
)

# Selection of the company (Mokka or Moica)
tk.Label(items_frame, text="Empresa").grid(row=0, column=0, padx=10, pady=5, sticky='w')
empresa_var = tk.StringVar(value="Mokka")
empresa_options = ["Mokka", "Moica"]
empresa_menu = tk.OptionMenu(items_frame, empresa_var, *empresa_options)
empresa_menu.grid(row=0, column=1, padx=10, pady=5, sticky='w')

# Add fields to the graphical interface
tk.Label(items_frame, text="CNPJ do Cliente").grid(row=1, column=0, padx=10, pady=5, sticky='w')
entrada_cnpj = tk.Entry(items_frame, width=50)
entrada_cnpj.grid(row=1, column=1, padx=10, pady=5, sticky='w')

# Selection of the person responsible for the quotation
tk.Label(items_frame, text="Responsável pela Cotação").grid(row=2, column=0, padx=10, pady=5, sticky='w')
responsavel_var = tk.StringVar()
responsavel_options = ["Weber Melo", "Thiago Velicev", "Giulia Armelin", "Letícia Casale"]
responsavel_menu = tk.OptionMenu(items_frame, responsavel_var, *responsavel_options)
responsavel_menu.grid(row=2, column=1, padx=10, pady=5, sticky='w')

# Selection of the client type (Final Client or Resale)
tk.Label(items_frame, text="Tipo de Cliente").grid(row=3, column=0, padx=10, pady=5, sticky='w')
cliente_tipo_var = tk.StringVar()
cliente_tipo_options = ["Cliente Final", "Revenda"]
cliente_tipo_menu = tk.OptionMenu(items_frame, cliente_tipo_var, *cliente_tipo_options)
cliente_tipo_menu.grid(row=3, column=1, padx=10, pady=5, sticky='w')

item_frames = []
adicionar_item()

tk.Label(items_frame, text="TOTAL DA COTAÇÃO").grid(row=100, column=0, padx=10, pady=5, sticky='w')
entrada_total_cotacao = tk.Entry(items_frame, width=20, state='readonly')
entrada_total_cotacao.grid(row=100, column=1, padx=10, pady=5, sticky='w')

tk.Label(items_frame, text="Prazo de Pagamento").grid(row=101, column=0, padx=10, pady=5, sticky='w')
entrada_prazo_pagamento = tk.Entry(items_frame, width=20)
entrada_prazo_pagamento.grid(row=101, column=1, padx=10, pady=5, sticky='w')

# Selection of the freight type
tk.Label(items_frame, text="Frete").grid(row=102, column=0, padx=10, pady=5, sticky='w')
frete_var = tk.StringVar(value="FOB")
tk.Radiobutton(items_frame, text="FOB", variable=frete_var, value="FOB").grid(row=102, column=1, padx=10, pady=5, sticky='w')
tk.Radiobutton(items_frame, text="CIF", variable=frete_var, value="CIF").grid(row=102, column=1, padx=100, pady=5, sticky='w')

# Spacing between fields and scrollbar
tk.Canvas(items_frame, width=cm).grid(row=0, column=2, rowspan=103)

botao_adicionar_item = tk.Button(items_frame, text="Adicionar Item", command=adicionar_item)
botao_adicionar_item.grid(row=104, column=0, columnspan=2, pady=10, sticky='w')

botao_gerar = tk.Button(items_frame, text="Gerar Cotação", command=gerar_cotacao)
botao_gerar.grid(row=105, column=0, columnspan=2, pady=20, sticky='w')

janela.mainloop()
